<a href="https://colab.research.google.com/github/SumathiGit/Natural-language-processing/blob/main/RNN_(predict_english_word_numbers).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [105]:
from fastai.text import *

In [106]:
bs=64

In [107]:
path = untar_data(URLs.HUMAN_NUMBERS)
path.ls()

[PosixPath('/root/.fastai/data/human_numbers/valid.txt'),
 PosixPath('/root/.fastai/data/human_numbers/train.txt')]

In [108]:
def readnums(d): return [', '.join(o.strip() for o in open(path/d).readlines())]

In [109]:
train_txt = readnums('train.txt'); train_txt[0][:80]

'one, two, three, four, five, six, seven, eight, nine, ten, eleven, twelve, thirt'

In [110]:
valid_txt = readnums('valid.txt'); valid_txt[0][-80:]

' nine thousand nine hundred ninety eight, nine thousand nine hundred ninety nine'

In [111]:
train = TextList(train_txt, path=path)
valid = TextList(valid_txt, path=path)

src = ItemLists(path=path, train=train, valid=valid).label_for_lm()
data = src.databunch(bs=bs)

In [112]:
train[0].text[:80]

'xxbos one , two , three , four , five , six , seven , eight , nine , ten , eleve'

In [113]:
len(data.valid_ds[0][0].data) #Total number of tokens 13017

13017

In [114]:
data.bptt, len(data.valid_dl) #bptt stands for back-propagation through time. This tells us how many steps of history(inputs) we are considering. in this case we hace 3 batches in our validation set and 70 tokens of numbers which are lines of text

(70, 3)

In [115]:
13017/70/bs            #2.90 >> comsidering as 3 batches

2.905580357142857

In [116]:
it = iter(data.valid_dl) #storing each batch in different variable (x and y)where x is a independent value and y is a dependent value like y is depending on x 
x1,y1 = next(it) #1
x2,y2 = next(it) #2
x3,y3 = next(it) #3
it.close()

In [117]:
x1 

tensor([[ 2, 19, 11,  ..., 36,  9, 19],
        [ 9, 19, 11,  ..., 24, 20,  9],
        [11, 27, 18,  ...,  9, 19, 11],
        ...,
        [20, 11, 20,  ..., 11, 20, 10],
        [20, 11, 20,  ..., 24,  9, 20],
        [20, 10, 26,  ..., 20, 11, 20]], device='cuda:0')

In [118]:
x1.numel()+x2.numel()+x3.numel()

13440

In [119]:
x1.shape, y1.shape      #First batch 64 lines of text and each line ha 70 tokens >> Tokenization = convert the text into a list of words

(torch.Size([64, 70]), torch.Size([64, 70]))

In [120]:
x2.shape, y2.shape

(torch.Size([64, 70]), torch.Size([64, 70]))

In [121]:
x3.shape, y3.shape

(torch.Size([64, 70]), torch.Size([64, 70]))

In [122]:
"""
Create a list of all possible unique words(duplicate words) in vocab
Replace words with the indexes.
Embedding matrix for each word.
Embedding matrix as an input to a neural network.
"""

'\nCreate a list of all possible unique words(duplicate words) in vocab\nReplace words with the indexes.\nEmbedding matrix for each word.\nEmbedding matrix as an input to a neural network.\n'

In [123]:
v = data.valid_ds.vocab 

In [124]:
v.itos #Mapping the integers to the string (int to string )

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 ',',
 'hundred',
 'thousand',
 'one',
 'two',
 'three',
 'four',
 'five',
 'six',
 'seven',
 'eight',
 'nine',
 'twenty',
 'thirty',
 'forty',
 'fifty',
 'sixty',
 'seventy',
 'eighty',
 'ninety',
 'ten',
 'eleven',
 'twelve',
 'thirteen',
 'fourteen',
 'fifteen',
 'sixteen',
 'seventeen',
 'eighteen',
 'nineteen',
 'xxfake']

In [125]:
# The xx which signifies that this is a start of document xx the , the next word is capitalized xx H and so on the string xx are directly mapped to vocab

In [126]:
x1[:,0]

tensor([ 2,  9, 11, 12, 13, 11, 10,  9, 10, 14, 19, 25, 19, 15, 16, 11, 19,  9,
        10,  9, 19, 25, 19, 11, 19, 11, 10,  9, 19, 20, 11, 26, 20, 23, 20, 20,
        24, 20, 11, 14, 11, 11,  9, 14,  9, 20, 10, 20, 35, 17, 11, 10,  9, 17,
         9, 20, 10, 20, 11, 20, 11, 20, 20, 20], device='cuda:0')

In [127]:
y1[:,0]

tensor([19, 19, 27, 10,  9, 12, 32, 19, 26, 10, 11, 15, 11, 10,  9, 15, 11, 19,
        26, 19, 11, 18, 11, 18,  9, 18, 21, 19, 10, 10, 20,  9, 11, 16, 11, 11,
        13, 11, 13,  9, 13, 14, 20, 10, 20, 11, 24, 11,  9,  9, 16, 17, 20, 10,
        20, 11, 24, 11, 19,  9, 19, 11, 11, 10], device='cuda:0')

In [128]:
v.itos[9], v.itos[11], v.itos[12], v.itos[13], v.itos[10]

(',', 'thousand', 'one', 'two', 'hundred')

In [129]:
#We can inspect each batches 

In [130]:
v.textify(x1[0])     #We can see in this code the initial xxbos indicates that this is the end of the word like mapping into vocab ,

'xxbos eight thousand one , eight thousand two , eight thousand three , eight thousand four , eight thousand five , eight thousand six , eight thousand seven , eight thousand eight , eight thousand nine , eight thousand ten , eight thousand eleven , eight thousand twelve , eight thousand thirteen , eight thousand fourteen , eight thousand fifteen , eight thousand sixteen , eight thousand seventeen , eight'

In [131]:
v.textify(y1[0])  #y values are just x shifted over by one

'eight thousand one , eight thousand two , eight thousand three , eight thousand four , eight thousand five , eight thousand six , eight thousand seven , eight thousand eight , eight thousand nine , eight thousand ten , eight thousand eleven , eight thousand twelve , eight thousand thirteen , eight thousand fourteen , eight thousand fifteen , eight thousand sixteen , eight thousand seventeen , eight thousand'

In [132]:
v.textify(x1[1])

', eight thousand forty six , eight thousand forty seven , eight thousand forty eight , eight thousand forty nine , eight thousand fifty , eight thousand fifty one , eight thousand fifty two , eight thousand fifty three , eight thousand fifty four , eight thousand fifty five , eight thousand fifty six , eight thousand fifty seven , eight thousand fifty eight , eight thousand fifty nine ,'

In [133]:
v.textify(x2[1])

'eight thousand sixty , eight thousand sixty one , eight thousand sixty two , eight thousand sixty three , eight thousand sixty four , eight thousand sixty five , eight thousand sixty six , eight thousand sixty seven , eight thousand sixty eight , eight thousand sixty nine , eight thousand seventy , eight thousand seventy one , eight thousand seventy two , eight thousand seventy three , eight thousand'

In [134]:
v.textify(x2[0]) #second batch continuous after x1[0]

'thousand eighteen , eight thousand nineteen , eight thousand twenty , eight thousand twenty one , eight thousand twenty two , eight thousand twenty three , eight thousand twenty four , eight thousand twenty five , eight thousand twenty six , eight thousand twenty seven , eight thousand twenty eight , eight thousand twenty nine , eight thousand thirty , eight thousand thirty one , eight thousand thirty two ,'

In [135]:
v.textify(x3[0])

'eight thousand thirty three , eight thousand thirty four , eight thousand thirty five , eight thousand thirty six , eight thousand thirty seven , eight thousand thirty eight , eight thousand thirty nine , eight thousand forty , eight thousand forty one , eight thousand forty two , eight thousand forty three , eight thousand forty four , eight thousand forty five , eight thousand forty six , eight'

In [136]:
v.textify(x3[-1])

'ninety , nine thousand nine hundred ninety one , nine thousand nine hundred ninety two , nine thousand nine hundred ninety three , nine thousand nine hundred ninety four , nine thousand nine hundred ninety five , nine thousand nine hundred ninety six , nine thousand nine hundred ninety seven , nine thousand nine hundred ninety eight , nine thousand nine hundred ninety nine xxbos eight thousand one , eight'

In [137]:
data.show_batch(ds_type=DatasetType.Valid)

idx,text
0,"thousand forty seven , eight thousand forty eight , eight thousand forty nine , eight thousand fifty , eight thousand fifty one , eight thousand fifty two , eight thousand fifty three , eight thousand fifty four , eight thousand fifty five , eight thousand fifty six , eight thousand fifty seven , eight thousand fifty eight , eight thousand fifty nine , eight thousand sixty , eight thousand sixty"
1,"eight , eight thousand eighty nine , eight thousand ninety , eight thousand ninety one , eight thousand ninety two , eight thousand ninety three , eight thousand ninety four , eight thousand ninety five , eight thousand ninety six , eight thousand ninety seven , eight thousand ninety eight , eight thousand ninety nine , eight thousand one hundred , eight thousand one hundred one , eight thousand one"
2,"thousand one hundred twenty four , eight thousand one hundred twenty five , eight thousand one hundred twenty six , eight thousand one hundred twenty seven , eight thousand one hundred twenty eight , eight thousand one hundred twenty nine , eight thousand one hundred thirty , eight thousand one hundred thirty one , eight thousand one hundred thirty two , eight thousand one hundred thirty three , eight thousand"
3,"three , eight thousand one hundred fifty four , eight thousand one hundred fifty five , eight thousand one hundred fifty six , eight thousand one hundred fifty seven , eight thousand one hundred fifty eight , eight thousand one hundred fifty nine , eight thousand one hundred sixty , eight thousand one hundred sixty one , eight thousand one hundred sixty two , eight thousand one hundred sixty three"
4,"thousand one hundred eighty three , eight thousand one hundred eighty four , eight thousand one hundred eighty five , eight thousand one hundred eighty six , eight thousand one hundred eighty seven , eight thousand one hundred eighty eight , eight thousand one hundred eighty nine , eight thousand one hundred ninety , eight thousand one hundred ninety one , eight thousand one hundred ninety two , eight thousand"


In [138]:
"""Single fully connected model"""

'Single fully connected model'

In [139]:
data = src.databunch(bs=bs, bptt=3)

In [140]:
x,y = data.one_batch()
x.shape,y.shape

(torch.Size([64, 3]), torch.Size([64, 3]))

In [141]:
nv = len(v.itos); nv

40

In [142]:
nh=64

In [143]:
def loss4(input,target): return F.cross_entropy(input, target[:,-1])
def acc4 (input,target): return accuracy(input, target[:,-1])

In [144]:
x[:,0]

tensor([13, 13, 10,  9, 18,  9, 11, 11, 13, 19, 16, 23, 24,  9, 12,  9, 13, 14,
        15, 11, 10, 22, 15,  9, 10, 14, 11, 16, 10, 28, 11,  9, 20,  9, 15, 15,
        11, 18, 10, 28, 23, 24,  9, 16, 10, 16, 19, 20, 12, 10, 22, 16, 17, 17,
        17, 11, 24, 10,  9, 15, 16,  9, 18, 11])

In [145]:
"""

    i_h: input to hidden
    h_h: hidden to hidden
    h_o: hidden to output
    bn: batchnorm
"""

'\n\n    i_h: input to hidden\n    h_h: hidden to hidden\n    h_o: hidden to output\n    bn: batchnorm\n'

In [146]:
class Model0(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv,nh)  # green arrow
        self.h_h = nn.Linear(nh,nh)     # brown arrow
        self.h_o = nn.Linear(nh,nv)     # blue arrow
        self.bn = nn.BatchNorm1d(nh)
        
    def forward(self, x):
        h = self.bn(F.relu(self.i_h(x[:,0])))
        if x.shape[1]>1:
            h = h + self.i_h(x[:,1])
            h = self.bn(F.relu(self.h_h(h)))
        if x.shape[1]>2:
            h = h + self.i_h(x[:,2])
            h = self.bn(F.relu(self.h_h(h)))
        return self.h_o(h)

In [147]:
learn = Learner(data, Model0(), loss_func=loss4, metrics=acc4)

In [148]:
learn.fit_one_cycle(6, 1e-4)

epoch,train_loss,valid_loss,acc4,time
0,3.699408,3.621562,0.085248,00:01
1,2.779298,2.813456,0.413143,00:01
2,2.178998,2.350663,0.447610,00:01
3,1.951658,2.187741,0.447840,00:01
4,1.870232,2.138943,0.447381,00:01
5,1.853232,2.132185,0.446921,00:01


In [149]:
# 45% accuracy

In [150]:
"""Multi fully connected model"""

'Multi fully connected model'

In [151]:
data = src.databunch(bs=bs, bptt=20)

In [152]:
x,y = data.one_batch()
x.shape,y.shape

(torch.Size([64, 20]), torch.Size([64, 20]))

In [153]:
class Model2(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv,nh)
        self.h_h = nn.Linear(nh,nh)
        self.h_o = nn.Linear(nh,nv)
        self.bn = nn.BatchNorm1d(nh)
        
    def forward(self, x):
        h = torch.zeros(x.shape[0], nh).to(device=x.device)
        res = []
        for i in range(x.shape[1]):
            h = h + self.i_h(x[:,i])
            h = F.relu(self.h_h(h))
            res.append(self.h_o(self.bn(h)))
        return torch.stack(res, dim=1)

In [154]:
learn = Learner(data, Model2(), metrics=accuracy)

In [155]:
learn.fit_one_cycle(10, 1e-4, pct_start=0.1)

epoch,train_loss,valid_loss,accuracy,time
0,3.933907,3.877102,0.005469,00:00
1,3.745763,3.623531,0.080611,00:00
2,3.514365,3.399290,0.133594,00:00
3,3.283687,3.219332,0.259091,00:00
4,3.078791,3.082876,0.284446,00:00
5,2.914142,2.990870,0.310440,00:00
6,2.793231,2.935348,0.334943,00:00
7,2.712178,2.905763,0.353622,00:00
8,2.663577,2.894475,0.359517,00:00
9,2.638464,2.892782,0.360511,00:00


In [156]:
"""Maintain state"""

'Maintain state'

In [157]:
class Model3(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv,nh)
        self.h_h = nn.Linear(nh,nh)
        self.h_o = nn.Linear(nh,nv)
        self.bn = nn.BatchNorm1d(nh)
        self.h = torch.zeros(bs, nh).cuda()
        
    def forward(self, x):
        res = []
        h = self.h
        for i in range(x.shape[1]):
            h = h + self.i_h(x[:,i])
            h = F.relu(self.h_h(h))
            res.append(self.bn(h))
        self.h = h.detach()
        res = torch.stack(res, dim=1)
        res = self.h_o(res)
        return res

In [158]:
learn = Learner(data, Model3(), metrics=accuracy)

In [159]:
learn.fit_one_cycle(20, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.594986,3.691166,0.091690,00:00
1,3.032453,2.772152,0.360440,00:00
2,2.297142,2.134304,0.316264,00:00
3,1.851244,2.104795,0.317472,00:00
4,1.637081,2.163412,0.316761,00:00
5,1.526264,2.067760,0.336861,00:00
6,1.404535,1.906699,0.425781,00:00
7,1.265511,1.873904,0.457031,00:00
8,1.137936,1.943355,0.480824,00:00
9,1.033857,2.017424,0.491051,00:00


In [160]:
"""nn.RNN"""

'nn.RNN'

In [161]:
class Model4(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv,nh)
        self.rnn = nn.RNN(nh,nh, batch_first=True)
        self.h_o = nn.Linear(nh,nv)
        self.bn = BatchNorm1dFlat(nh)
        self.h = torch.zeros(1, bs, nh).cuda()
        
    def forward(self, x):
        res,h = self.rnn(self.i_h(x), self.h)
        self.h = h.detach()
        return self.h_o(self.bn(res))

In [162]:
learn = Learner(data, Model4(), metrics=accuracy)

In [163]:
learn.fit_one_cycle(20, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.483227,3.117459,0.331960,00:00
1,2.778485,2.227514,0.460653,00:00
2,2.123126,2.009571,0.397230,00:00
3,1.769151,2.077395,0.316690,00:00
4,1.586660,1.937464,0.394247,00:00
5,1.447810,1.762954,0.488707,00:00
6,1.282328,1.744812,0.495242,00:00
7,1.110538,1.693838,0.534446,00:00
8,0.939928,1.420049,0.583523,00:00
9,0.788167,1.377960,0.602770,00:00


In [164]:
"""2-layer GRU"""

'2-layer GRU'

In [ ]:
"""
GRU -A Gated Recurrent Unit (GRU), as its name suggests, is a variant of the RNN architecture,
and uses gating mechanisms to control and manage the flow of information between cells in the neural network. 
      !.Update gate
      2.Reset Gate   
these gates are responsible for deciding which portions of the previous hidden state 
are to be combined with the current input to propose a new hidden state.
"""

In [165]:
class Model5(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv,nh)
        self.rnn = nn.GRU(nh, nh, 2, batch_first=True)
        self.h_o = nn.Linear(nh,nv)
        self.bn = BatchNorm1dFlat(nh)
        self.h = torch.zeros(2, bs, nh).cuda()
        
    def forward(self, x):
        res,h = self.rnn(self.i_h(x), self.h)
        self.h = h.detach()
        return self.h_o(self.bn(res))    #Applying batch norm without loop which makes the network faster

In [166]:
learn = Learner(data, Model5(), metrics=accuracy)

In [167]:
learn.fit_one_cycle(10, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,2.739483,2.349902,0.445526,00:00
1,1.586334,1.389369,0.643821,00:00
2,0.758286,1.103908,0.785724,00:00
3,0.368020,0.897834,0.839702,00:00
4,0.186388,0.932994,0.831108,00:00
5,0.098814,1.003976,0.831250,00:00
6,0.056015,1.064659,0.822940,00:00
7,0.034275,1.171127,0.820952,00:00
8,0.022505,1.170534,0.813139,00:00
9,0.016303,1.175788,0.811506,00:00


In [ ]:
81% accuracy